Autor: João Felipe B. C. Carvalho
O programa se trata de um controlador de grafo de atividades. É possível incluir um grafo via arquivo "csv" ou utilizar o prompt para montar. Caso utilize o csv suba o arquivo na parte esquerda do colab no ícone de pasta.

In [2]:
import numpy as np
import csv

def isFinais(dPertCpm):
  naoFinais = []
  for key in dPertCpm.keys():
    for key2 in dPertCpm.keys():
      if key in dPertCpm[key2]["precedente"]:
        naoFinais.append(key)
        break

  a = set(dPertCpm.keys())
  b = set(naoFinais)
  return list(a.symmetric_difference(b))

def inclui_csv(cInput):
  dPertCpm = {}
  infoAdd = {}
  with open(cInput+".csv", "r") as arquivo_csv:
    leitor = csv.reader(arquivo_csv, delimiter=";")
    cont = 1
    for coluna in leitor:
      if coluna[2] == "-":
        precedente = ['']
      else:
        precedente  = coluna[2].split(",")
      dPertCpm.update({str(cont): {"atividade": coluna[0], "duração": float(coluna[1]), "precedente": precedente, "tempIniCedo": 0, "tempoFimCedo": 0, "tempoIniTard": 0, "tempoFimTard": 0, "folga": 0}})
      cont += 1  
  infoAdd = {"inicio" : 1, "fim" : 0,"finais" : isFinais(dPertCpm)}
  return dPertCpm, infoAdd

  
def inclui_tab():
  dPertCpm = {}
  cont = 1
  atv = ""
  duracao = 0
  precedente = []
  atv, duracao = input("Linha " + str(cont) + ". Inclua os dados de Atividade e  Duração separados por vírgula:").split(",")
  precedente  = input("Precedentes(separados por vírgula):").split(",")
  dPertCpm.update({str(cont): {"atividade": atv, "duração": float(duracao), "precedente": precedente, "tempIniCedo": 0, "tempoFimCedo": 0, "tempoIniTard": 0, "tempoFimTard": 0, "folga": 0}})
  while True:
    if input("Incluir mais uma linha?\n1 - Sim\n2 - Não\nR:") == "2":
      break
    cont += 1 
    atv, duracao = input("Linha " + str(cont) + ". Inclua os dados de Atividade e Duração, separados por vírgula:").split(",")
    precedente  = input("Precedentes(separados por vírgula):").split(",")
    dPertCpm.update({str(cont): {"atividade": atv, "duração": float(duracao), "precedente": precedente, "tempIniCedo": 0, "tempoFimCedo": 0, "tempoIniTard": 0, "tempoFimTard": 0, "folga": 0}})
  
  infoAdd = {"inicio" : 1, "fim" : 0,"finais" : isFinais(dPertCpm)}

  
  return dPertCpm, infoAdd

def calculoTempIni(dPertCpm,dInfoAdd):
  vertIni = "0"
  for vertIni in dPertCpm.keys():
    if len(dPertCpm[vertIni]["precedente"]) == 1 and len(dPertCpm[vertIni]["precedente"][0]) == 0  :
      dPertCpm[vertIni]["tempIniCedo"] = 1
      dPertCpm[vertIni]["tempoFimCedo"] = dPertCpm[vertIni]["duração"] 
    elif len(dPertCpm[vertIni]["precedente"]) == 1 and len(dPertCpm[vertIni]["precedente"][0]) > 0 :
      precedente = dPertCpm[vertIni]["precedente"][0] 
      dPertCpm[vertIni]["tempIniCedo"] = dPertCpm[precedente]["tempoFimCedo"] + 1
      dPertCpm[vertIni]["tempoFimCedo"] = dPertCpm[vertIni]["duração"] + dPertCpm[vertIni]["tempIniCedo"] - 1
    else:
      maior = dPertCpm[dPertCpm[vertIni]["precedente"][0]]["tempoFimCedo"]
      for prc in dPertCpm[vertIni]["precedente"]:
        if dPertCpm[prc]["tempoFimCedo"] > maior:
          maior = dPertCpm[prc]["tempoFimCedo"]
      dPertCpm[vertIni]["tempIniCedo"] = maior + 1
      dPertCpm[vertIni]["tempoFimCedo"] = dPertCpm[vertIni]["duração"] + dPertCpm[vertIni]["tempIniCedo"] - 1

  vertIni = "0"
  maior = dPertCpm[dInfoAdd["finais"][0]]["tempoFimCedo"]
  for vertIni in dInfoAdd["finais"]:
    if dPertCpm[vertIni]["tempoFimCedo"] > maior:
          maior = dPertCpm[vertIni]["tempoFimCedo"]
  dInfoAdd["fim"] = maior

  return dPertCpm, dInfoAdd
      
def calculoTempFim(dPertCpm, dInfoAdd):
  aPrecedentes = []
  aFinais = dInfoAdd["finais"]
  nCont = 0
  for fim in dInfoAdd["finais"]:
    dPertCpm[fim]["tempoIniTard"] = dInfoAdd["fim"] - (dPertCpm[fim]["duração"]  - 1)
    
    dPertCpm[fim]["tempoFimTard"] = dInfoAdd["fim"] 
    aPrecedentes.append([dPertCpm[fim]["precedente"], dPertCpm[fim]["tempoIniTard"]])

  while len(aPrecedentes) > 0 :
    for ind in range(len(aPrecedentes[nCont][0])):
      precedente = aPrecedentes[nCont][0][ind]
      if precedente != '':
        if  dPertCpm[precedente]["tempoFimTard"] == 0:
          dPertCpm[precedente]["tempoFimTard"] = aPrecedentes[nCont][1] - 1
          dPertCpm[precedente]["tempoIniTard"] = dPertCpm[precedente]["tempoFimTard"] - (dPertCpm[precedente]["duração"] - 1)
          aPrecedentes.append([dPertCpm[precedente]["precedente"], dPertCpm[precedente]["tempoIniTard"]] )
        elif dPertCpm[precedente]["tempoFimTard"] > (aPrecedentes[nCont][1] - 1):
          dPertCpm[precedente]["tempoFimTard"] = aPrecedentes[nCont][1] - 1
          dPertCpm[precedente]["tempoIniTard"] = dPertCpm[precedente]["tempoFimTard"] - (dPertCpm[precedente]["duração"] - 1)
          aPrecedentes.append([dPertCpm[precedente]["precedente"], dPertCpm[precedente]["tempoIniTard"]] )
    aPrecedentes.pop(nCont)


  return dPertCpm

def getWayCritic(dPertCpm):
  aCritic = []
  for key in dPertCpm.keys():
    if dPertCpm[key]["folga"] == 0:
      aCritic.append(key)
  return aCritic

def setFolga(dPertCpm):
  for key in dPertCpm.keys():
    dPertCpm[key]["folga"] = abs(dPertCpm[key]["tempIniCedo"] - dPertCpm[key]["tempoIniTard"] )
  
  return dPertCpm


cEscolha = input("Deseja incluir o diagrama manualmente ou através de arquivo .csv?\n1 - Manualmente\n2 - CSV\nEscolha:")
if cEscolha == "1":
  dPertCpmMaster, dInfoAdd = inclui_tab()
else:
  cInput= input("Informe o nome do arquivo:")
  dPertCpmMaster, dInfoAdd = inclui_csv(cInput) 

#inclui_tab()
dPertCpmMaster, dInfoAdd = calculoTempIni(dPertCpmMaster, dInfoAdd)
dPertCpmMaster = calculoTempFim(dPertCpmMaster, dInfoAdd)
dPertCpmMaster = setFolga(dPertCpmMaster)
aCritico = getWayCritic(dPertCpmMaster)
print("O caminho crítico do grafo de atividades é:")
print(aCritico)
print("Grafo de atividades:")

for cKey in dPertCpmMaster.keys():
    print(dPertCpmMaster[cKey])

print("Informações das atividades:")
print(dInfoAdd)


Deseja incluir o diagrama manualmente ou através de arquivo .csv?
1 - Manualmente
2 - CSV
Escolha:2
Informe o nome do arquivo:teste
O caminho crítico do grafo de atividades é:
['1', '2', '3', '5', '6', '10', '12', '14']
Grafo de atividades:
{'atividade': 'A', 'duração': 2.0, 'precedente': [''], 'tempIniCedo': 1, 'tempoFimCedo': 2.0, 'tempoIniTard': 1.0, 'tempoFimTard': 2.0, 'folga': 0.0}
{'atividade': 'B', 'duração': 4.0, 'precedente': ['1'], 'tempIniCedo': 3.0, 'tempoFimCedo': 6.0, 'tempoIniTard': 3.0, 'tempoFimTard': 6.0, 'folga': 0.0}
{'atividade': 'C', 'duração': 10.0, 'precedente': ['2'], 'tempIniCedo': 7.0, 'tempoFimCedo': 16.0, 'tempoIniTard': 7.0, 'tempoFimTard': 16.0, 'folga': 0.0}
{'atividade': 'D', 'duração': 6.0, 'precedente': ['3'], 'tempIniCedo': 17.0, 'tempoFimCedo': 22.0, 'tempoIniTard': 21.0, 'tempoFimTard': 26.0, 'folga': 4.0}
{'atividade': 'E', 'duração': 4.0, 'precedente': ['3'], 'tempIniCedo': 17.0, 'tempoFimCedo': 20.0, 'tempoIniTard': 17.0, 'tempoFimTard': 20.0, 